In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

### Activation
---
- 유저 당 세션 수
- 이벤트 참여수 (이벤트 페이지 조회 수)
- 이벤트를 발생 시킨 특정 세션 수(이벤트 페이지 조회 수)
- 특정 이벤트를 발생 시킨 특정 유저 수

### Revenue
---
- 총 매출 (Total Revenue)
- 구매 전환율 (Purchase Conversion Rate)
- 고객당 평균 매출 (Average Revenue per User, ARPU)
- 세션당 평균 매출 (Average Revenue per Session)
- 가장 많이 구매한 카테고리 아이템
- 주문 빈도 - Order Frequency
- 평균 주문 금액 - (Average Order Value, AOV)

In [2]:
df=pd.read_csv('processed_data.csv')
df.head()

,Unnamed: 0,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,year,month,day,hour,main_category,sub_category,sub_sub_category
0,0,2020-09-24 11:57:06+00:00,view,1996170,2144415922528452715,electronics.telephone,NaN,31.90,1515915625519388267,LJuJVLEjPT,2020,9,24,11,electronics,telephone,NaN
1,1,2020-09-24 11:57:26+00:00,view,139905,2144415926932472027,computers.components.cooler,zalman,17.16,1515915625519380411,tdicluNnRY,2020,9,24,11,computers,components,cooler
2,2,2020-09-24 11:57:27+00:00,view,215454,2144415927158964449,NaN,NaN,9.81,1515915625513238515,4TMArHtXQy,2020,9,24,11,NaN,NaN,NaN
3,3,2020-09-24 11:57:33+00:00,view,635807,2144415923107266682,computers.peripherals.printer,pantum,113.81,1515915625519014356,aGFYrNgC08,2020,9,24,11,computers,peripherals,printer
4,4,2020-09-24 11:57:36+00:00,view,3658723,2144415921169498184,NaN,cameronsino,15.87,1515915625510743344,aa4mmk0kwQ,2020,9,24,11,NaN,NaN,NaN


In [5]:
# 날짜별 집계를 위한 설정
df['event_date']=pd.to_datetime(df['event_time']).dt.date
gr=df.groupby('event_date')

In [100]:
for name,groups in gr:
    print(name)
    print(groups.head())

2020-09-24
                event_time event_type  product_id          category_id  \
0  2020-09-24 11:57:06 UTC       view     1996170  2144415922528452715   
1  2020-09-24 11:57:26 UTC       view      139905  2144415926932472027   
2  2020-09-24 11:57:27 UTC       view      215454  2144415927158964449   
3  2020-09-24 11:57:33 UTC       view      635807  2144415923107266682   
4  2020-09-24 11:57:36 UTC       view     3658723  2144415921169498184   

                   category_code        brand   price              user_id  \
0          electronics.telephone          NaN   31.90  1515915625519388267   
1    computers.components.cooler       zalman   17.16  1515915625519380411   
2                            NaN          NaN    9.81  1515915625513238515   
3  computers.peripherals.printer       pantum  113.81  1515915625519014356   
4                            NaN  cameronsino   15.87  1515915625510743344   

  user_session  event_date  
0   LJuJVLEjPT  2020-09-24  
1   tdicluNnRY  2

In [107]:
unique_view_sessions=gr.apply(lambda x: x.loc[x['event_type']=='view','user_session'].nunique()) #1
unique_view_sessions

event_date
2020-09-24    1440
2020-09-25    2627
2020-09-26    2157
2020-09-27    2340
2020-09-28    2902
              ... 
2021-02-24    3239
2021-02-25    3095
2021-02-26    3256
2021-02-27    2834
2021-02-28    3042
Length: 158, dtype: int64

In [133]:
Activation = gr.agg(
    
    sessions_per_user=('user_session', lambda x: x.count() / gr.get_group(x.name)['user_id'].nunique()),
    view_count=('event_type', lambda x: (x == 'view').sum()),
    cart_count=('event_type', lambda x: (x == 'cart').sum()),
    purchase_count=('event_type', lambda x: (x == 'purchase').sum()),
    unique_view_sessions = ('user_session',lambda x: x[df['event_type']=='view'].nunique()), #1
    unique_cart_sessions = ('user_session',lambda x: x[df['event_type']=='cart'].nunique()),
    unique_purchase_sessions = ('user_session',lambda x: x[df['event_type']=='purchase'].nunique()),
    unique_view_users = ('user_id',lambda x: x[df['event_type']=='view'].nunique()),
    unique_cart_users = ('user_id',lambda x: x[df['event_type']=='cart'].nunique()),
    unique_purchase_users = ('user_id',lambda x: x[df['event_type']=='purchase'].nunique())
    
                    ).reset_index()

Activation.head()

,event_date,sessions_per_user,view_count,cart_count,purchase_count,unique_view_sessions,unique_cart_sessions,unique_purchase_sessions,unique_view_users,unique_cart_users,unique_purchase_users
0,2020-09-24,1.667158,2082,99,84,1440,83,58,1355,81,56
1,2020-09-25,1.730314,3852,240,149,2627,199,109,2445,193,105
2,2020-09-26,1.705911,3169,177,119,2157,132,76,2028,132,73
3,2020-09-27,1.760402,3569,172,110,2340,142,72,2183,137,70
4,2020-09-28,1.725476,4255,259,200,2902,201,123,2726,197,118


In [120]:
DAU=gr['user_id'].nunique()
purchase_count = gr['event_type'].apply(lambda x: (x=='purchase').sum())
purchase_conversion_rate = (purchase_count/DAU).round(3) #1
purchase_conversion_rate.head()

event_date
2020-09-24    0.062
2020-09-25    0.061
2020-09-26    0.059
2020-09-27    0.050
2020-09-28    0.073
dtype: float64

In [129]:
total_purchase_amount = gr.apply(lambda x: x.loc[x['event_type'] == 'purchase', 'price'].sum())
ARPU = total_purchase_amount/DAU
ARPU.head(2).round(2) #2

event_date
2020-09-24    7.04
2020-09-25    5.31
dtype: float64

In [148]:
high_brand_daily  = gr.apply(lambda x: x[x['event_type'] == 'purchase'].groupby('brand')['price'].sum().idxmax())
high_brand_daily

event_date
2020-09-24        asus
2020-09-25         amd
2020-09-26       epson
2020-09-27    gigabyte
2020-09-28       epson
                ...   
2021-02-24         msi
2021-02-25    gigabyte
2021-02-26         msi
2021-02-27         msi
2021-02-28         msi
Length: 158, dtype: object

In [45]:
Activation = gr.agg(
    sessions_per_user=('user_session', lambda x: (x.count() / gr['user_id'].nunique())),
    view_count=('event_type', lambda x: (x == 'view').sum()),
    cart_count=('event_type', lambda x: (x == 'cart').sum()),
    purchase_count=('event_type', lambda x: (x == 'purchase').sum())).reset_index()
Activation

,event_date,sessions_per_user,view_count,cart_count,purchase_count
0,2020-09-24,1.667158,2082,99,84
1,2020-09-25,3.122975,3852,240,149
2,2020-09-26,2.550074,3169,177,119
3,2020-09-27,2.835052,3569,172,110
4,2020-09-28,3.471281,4255,259,200
...,...,...,...,...,...
153,2021-02-24,4.273196,5098,413,292
154,2021-02-25,4.158321,4988,411,248
155,2021-02-26,4.318115,5173,416,275
156,2021-02-27,3.632548,4385,325,223


In [150]:
Revenue = gr.agg(
    
    total_purchase_amount = ('price', lambda x: x[df['event_type']=='purchase'].sum()),
    purchase_conversion_rate = ('event_type',lambda x: (x=='purchase').sum()/
                                gr.get_group(x.name)['user_id'].nunique()), #1
    ARPU = ('price',lambda x: (x[df['event_type']=='purchase'].sum()/
                               gr.get_group(x.name)['user_id'].nunique()).round(2)), #2
    ARPS = ('price',lambda x: (x[df['event_type']=='purchase'].sum()/
                               gr.get_group(x.name)['user_session'].nunique()).round(2)),
    high_brand_daily = ('price', lambda x: x[df['event_type']=='purchase'].groupby(df['brand']).sum().idxmax()) #3
                            
                ).reset_index()

Revenue.head()

,event_date,total_purchase_amount,purchase_conversion_rate,ARPU,ARPS,high_brand_daily
0,2020-09-24,9563.67,0.061856,7.04,6.61,asus
1,2020-09-25,13025.72,0.060792,5.31,4.94,amd
2,2020-09-26,11821.97,0.058621,5.82,5.46,epson
3,2020-09-27,7910.53,0.050297,3.62,3.37,gigabyte
4,2020-09-28,15120.82,0.073206,5.53,5.19,epson


In [3]:
df['event_date']=pd.to_datetime(df['event_time']).dt.date
gr=df.groupby('event_date')

Activation = gr.agg(
    
    sessions_per_user=('user_session', lambda x: (x.count() / gr.get_group(x.name)['user_id'].nunique()).round(2)),
    view_count=('event_type', lambda x: (x == 'view').sum()),
    cart_count=('event_type', lambda x: (x == 'cart').sum()),
    purchase_count=('event_type', lambda x: (x == 'purchase').sum()),
    unique_view_sessions = ('user_session',lambda x: x[df['event_type']=='view'].nunique()), 
    unique_cart_sessions = ('user_session',lambda x: x[df['event_type']=='cart'].nunique()),
    unique_purchase_sessions = ('user_session',lambda x: x[df['event_type']=='purchase'].nunique()),
    unique_view_users = ('user_id',lambda x: x[df['event_type']=='view'].nunique()),
    unique_cart_users = ('user_id',lambda x: x[df['event_type']=='cart'].nunique()),
    unique_purchase_users = ('user_id',lambda x: x[df['event_type']=='purchase'].nunique())
    
                    ).reset_index()

Revenue = gr.agg(
    
    total_purchase_amount = ('price', lambda x: (x[df['event_type']=='purchase'].sum()).round(2)),
    purchase_conversion_rate = ('event_type',lambda x: (x=='purchase').sum()/
                                gr.get_group(x.name)['user_id'].nunique()), 
    ARPU = ('price',lambda x: (x[df['event_type']=='purchase'].sum()/
                               gr.get_group(x.name)['user_id'].nunique()).round(2)), 
    ARPS = ('price',lambda x: (x[df['event_type']=='purchase'].sum()/
                               gr.get_group(x.name)['user_session'].nunique()).round(2)),
    high_brand_daily = ('price', lambda x: x[df['event_type']=='purchase'].groupby(df['brand']).sum().idxmax()) 
                            
                ).reset_index()

merge_df=pd.merge(Activation, Revenue, on='event_date', how='inner')
merge_df.head()

,event_date,sessions_per_user,view_count,cart_count,purchase_count,unique_view_sessions,unique_cart_sessions,unique_purchase_sessions,unique_view_users,unique_cart_users,unique_purchase_users,total_purchase_amount,purchase_conversion_rate,ARPU,ARPS,high_brand_daily
0,2020-09-24,1.67,2082,99,84,1440,83,58,1355,81,56,9563.67,0.061856,7.04,6.61,asus
1,2020-09-25,1.73,3852,240,149,2627,199,109,2445,193,105,13025.72,0.060792,5.31,4.94,amd
2,2020-09-26,1.71,3169,177,119,2157,132,76,2028,132,73,11821.97,0.058621,5.82,5.46,epson
3,2020-09-27,1.76,3569,172,110,2340,142,72,2183,137,70,7910.53,0.050297,3.62,3.37,gigabyte
4,2020-09-28,1.73,4255,259,200,2902,201,123,2726,197,118,15120.82,0.073206,5.53,5.19,epson


### 코딩 관련 어려웠고, GPT 도움 받은 점:
1. sessions_per_user= gr['user_session'].count()/gr['user_id'].nunique() 이렇게 구하면 쉽지만, Activation의 gr 내에서 정의하고 싶어서 시도 여러번 했으나 같은 결과가 안나와서 gpt에게 물어봤습니다.
```
agg 함수 사용:

x는 현재 그룹의 user_session 컬럼의 시리즈입니다.
x.name은 현재 그룹의 키, 즉 event_date입니다.
예를 들어, x.name이 2020-09-24이면, x는 2020-09-24에 해당하는 user_session 컬럼의 시리즈입니다.
gr.get_group(x.name)는 event_date가 2020-09-24인 전체 그룹을 반환합니다.
lambda 함수 내부에서의 동작:

x.count()는 현재 그룹의 user_session 컬럼 값의 개수를 셉니다.
gr.get_group(x.name)['user_id'].nunique()는 해당 그룹(날짜)의 고유 user_id 개수를 셉니다.
```